# HRRR NOAA Temperature data

EL siguiente código emplea el paquete [*Herbie*](https://herbie.readthedocs.io/en/stable/) para descargar la temperatura obtenida a partir del satélite [NOAH HRRR](https://rapidrefresh.noaa.gov/hrrr/). Solo para complementar la información de los datos descargados de Sentinel-2

## Imports

In [ ]:
from herbie import Herbie

from pathlib import Path
import pandas as pd
import geopy.distance as distance
from tqdm import tqdm
import json

## Parámetros

In [ ]:
DATA_DIR = Path.cwd().resolve() / "data"
SENTINEL_DATA_DIR = DATA_DIR / "sentinel"

## Funciones

In [ ]:
def get_bounding_box(latitude, longitude, meter_buffer=50000):
    """
    Dada una latitud, longitud y buffer en metros, devuelve una superficie rectangular alrededor de las coordenadas proporcionadas en
    cada una de las direcciones cardinales

    Devuelve una lista de [minx, miny, maxx, maxy]
    """
    distance_search = distance.distance(meters=meter_buffer)

    # calcula la lat/long basada en la distancia en tierra
    # cada orientación corresponde a una dirección cardinal (sur, oeste, norte, y este)
    min_lat = distance_search.destination((latitude, longitude), bearing=180)[0]
    min_long = distance_search.destination((latitude, longitude), bearing=270)[1]
    max_lat = distance_search.destination((latitude, longitude), bearing=0)[0]
    max_long = distance_search.destination((latitude, longitude), bearing=90)[1]

    return [min_long, min_lat, max_long, max_lat]

## Extracción

In [ ]:
metadata = pd.read_csv(DATA_DIR / "metadata.csv")

In [ ]:
sentinel_data = json.load(open(SENTINEL_DATA_DIR / "selected_items.txt"))

In [ ]:
train_subset = metadata[metadata.uid.isin(sentinel_data.keys())]

In [ ]:
temp_dict = {} # Registra las temperaturas medias calculadas

# Constantes para crear la URL completa
forecast_hour = 0   # Offset del ciclo temporal
product = 'sfc' # Superficie 2D

i=0 # Contador

for row in tqdm(train_subset.itertuples(), total=len(train_subset)):
    pass
    # Se comprueba que no existe el elemento ( en caso de cargar un archivo con temperaturas de ejecuciones anteriores)
    if temp_dict.get(row.uid) == None:
        date = row.date
        
        # recorte de 2000 m alrededor de las coordenadas de medición
        [min_long, min_lat, max_long, max_lat] = get_bounding_box(row.latitude, row.longitude, 2000)
        try:
            H = Herbie(
                date,  # fecha
                model="hrrr",  # nombre del modelo
                product=product,  # nombre del producto del modelo
                fxx=forecast_hour,  # forecast
            )
            ds = H.xarray(":TMP:surface", remove_grib=True) # Lee la temperatura en la superficie
            
            # Cálculo de la temperatura media dentro del recorte y almacenamiento
            temp_dict[row.uid] = float(ds.t.where(ds.latitude >= min_lat).where(ds.latitude <= max_lat).where(ds.longitude <= max_long+360).where(ds.longitude >= min_long+360).mean(skipna=True).values)
        except:
            # Se asigna menos 1 a las temperaturas que no se han podido descargar
            temp_dict[row.uid] = -1
        
        # Copia de seguridad cada 100 iteraciones
        if i % 100 == 0 and i >= 100:
            with open(SENTINEL_DATA_DIR / "temperatures.txt", "w") as f:
                json.dump(temp_dict, f)
        i += 1


## Referencias

[1] Brian K. Blaylock. Herbie: Retrieve NWP Model Data: https://herbie.readthedocs.io/en/stable/

[2] Global Systems Laboratory. The High-Resolution Rapid Refresh (HRRR): https://rapidrefresh.noaa.gov/hrrr/